In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [23]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

def extract_game_df_from_url(url_list):
    """
    Initializes a DataFrame with the same structure as `dsk_sample_for_gpt`
    and populates the `oppo_cards_in_hand` column based on the HTML from URLs.

    Args:
        url_list (list of str): List of URLs pointing to game states.

    Returns:
        pd.DataFrame: DataFrame with the same structure as `dsk_sample_for_gpt`.
    """
    # Load the column structure from the sample file
    with open("../data/win_prob_model_resources/model_cols.txt", "r") as file:
        columns = [line.strip() for line in file.readlines()]
    
    df = pd.DataFrame(columns=columns)

    # Set up Selenium WebDriver
    service = Service('path_to_chromedriver')  # Update with your ChromeDriver path
    options = Options()
    options.add_argument("--headless")  # Run in headless mode
    driver = webdriver.Chrome(service=service, options=options)

    for idx, url in enumerate(url_list):
        try:
            # Fetch the fully rendered HTML using Selenium
            driver.get(url)
            time.sleep(5)  # Allow time for dynamic content to load

            # Get the page source
            rendered_html = driver.page_source
            
            # Parse the rendered HTML
            soup = BeautifulSoup(rendered_html, 'html.parser')
            
            # Locate the opponent's hand area
            oppo_area = soup.find('div', class_='upper-player')
            if oppo_area is None:
                print(f"No 'upper-player' div found for URL: {url}")
                continue  # Skip to the next URL if structure is missing

            hand_area = oppo_area.find('div', class_='hand-area')
            if hand_area is None:
                print(f"No 'hand-area' div found in 'upper-player' for URL: {url}")
                continue  # Skip to the next URL if structure is missing

            # Count the number of cards in the opponent's hand
            oppo_hand_count = len(hand_area.find_all('div', class_='card-container'))

            # Populate the DataFrame
            df = df.append({
                'game_id': f'game_{idx + 1}',  # Mock game_id
                'turn': idx + 1,               # Mock turn number
                'oppo_cards_in_hand': oppo_hand_count,
                # Other columns default to NaN
            }, ignore_index=True)

        except Exception as e:
            print(f"Error fetching or parsing URL {url}: {e}")
            continue

    driver.quit()  # Close the browser once all URLs are processed
    return df


In [24]:
extract_game_df_from_url(["https://www.17lands.com/history/120aad874c1b4f82b7831f879a13c396/2/0"])

NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location
